<a href="https://colab.research.google.com/github/LinarKulinar/HPC_SSAU/blob/master/Lab1_Vector_summation/Lab1_Vector_summation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Высокопроизводительные вычисления.**

**Лабораторная 1. Сложение векторов на CPU и GPU.**

Самарский университет

В данном блокноте реализован алгоритм сложения векторов для CPU и для GPU.

Для выполнения кода на C на GPU используется библиотека PyCuda.

Данный код запущен в среде Google Colaboratory.

In [1]:
! pip install pycuda

     |████████████████████████████████| 1.7 MB 5.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 66 kB 4.7 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626633 sha256=6736a2d889f29037e50618523b8cbe1a61912a5f0870a9a95ee9d790d4e18dd0
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2021.2.9-py2.py3-none-any.whl size=62370 sha256=116bb471fa75df9e5e9b50797d5ab0fb650da8e04da8afc2d648f1256eb65d76
  Stored in directory: /root/.cache/pip/wheels/41/b9/6e/94bb014f6484b15ec77e7877f3a227609481ffd98db364504d
Successfully built pycuda pytools


In [2]:
!nvidia-smi

Sun Jan  9 15:51:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import numpy as np
import time

import pycuda.gpuarray as gpuarray
import pycuda.autoinit
from  pycuda import  driver
from pycuda.compiler import SourceModule

Определяем функцию сложения векторов на C, которая будет выполняться на GPU

In [4]:
mod = SourceModule("""
  __global__ void vec_sum_gpu(double *x, double *y, double *c_gpu, const int n) {  
      int idx = threadIdx.x + (blockIdx.x*blockDim.x);
      c_gpu[idx] = x[idx] + y[idx];
  }    
""")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  


Определяем функцию сложения векторов на python, которая будет выполняться на CPU

In [5]:
def vec_sum_cpu(x, y, n):
  c = npa.zeros(n)
  for i in range(n):
    c[i] = x[i] + y[i]
  return c

Вводим входные параметры задачи - размеры складываемых массивов

In [6]:
def generate_data():a
  n = int(input('Введите размер векторов = '))

  x = np.random.randn(n).astype(np.float)
  y = np.random.randn(n).astype(np.float)
  print('Сгенерированные массивы: \n', x, '\n', y)
  return x, y, n

In [7]:
x, y, n = generate_data()

Введите размер векторов = 10000
Сгенерированные массивы: 
 [ 0.09064856  0.0643606   1.98671676 ... -0.02244204  0.02321446
 -1.41029957] 
 [-0.32766814 -0.58891653  0.99948161 ...  1.75017245  0.40300409
  1.38901357]


Вычисляем на CPU, замеряя время

In [8]:
cpu_start = time.time()
result_cpu = vec_sum_cpu(x,y, n)
cpu_time = time.time() - cpu_startф
print('Результирующий вектор: ', result_cpu)
print('Время на CPU: ', round(cpu_time,4))

Результирующий вектор:  [-0.23701958 -0.52455593  2.98619838 ...  1.72773041  0.42621855
 -0.021286  ]
Время на CPU:  0.0105


Вычисляем на GPU, замеряя время

In [9]:
# т.к. массивы являются одномерными
block_size = 128
block = (block_size, 1, 1)
grid = (int((n + block_size - 1) / block_size), 1)
block, grid

((128, 1, 1), (79, 1))

In [10]:
# c_gpu = np.ones(n).astype(np.float)
result_gpu = gpuarray.zeros((n), dtype=np.double)
result_gpu  = result_gpu.get()

vec_sum_gpu = mod.get_function("vec_sum_gpu")

gpu_start = time.time()
vec_sum_gpu(driver.In(x), driver.In(y), driver.Out(result_gpu), np.int32(n), block = block, grid = grid)
driver.Context.synchronize()
gpu_time = time.time() - gpu_start

print('Результирующий вектор: ', result_gpu)
print('Время на GPU: ', round(gpu_time,4))

Результирующий вектор:  [-0.23701958 -0.52455593  2.98619838 ...  1.72773041  0.42621855
 -0.021286  ]
Время на GPU:  0.0009


/usr/local/lib/python3.7/dist-packages/pycuda/elementwise.py:82: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  no_extern_c=True,


Выводим результаты

In [12]:
print('Vector size: ', n)
print('Results converge;  acceleration: ', round(cpu_time/gpu_time, 4)) if np.allclose(result_gpu, result_cpu) else print('Results do not converge')

print('Time on GPU: ', round(gpu_time,4))
print('Time on CPU: ', round(cpu_time,4))

Vector size:  10000
Results converge;  acceleration:  11.7236
Time on GPU:  0.0009
Time on CPU:  0.0105
